In [1]:
#!unzip -nq /datasets/minispeechcommands/msc-train.zip
#!unzip -nq /datasets/minispeechcommands/msc-val.zip
#!unzip -nq /datasets/minispeechcommands/msc-test.zip

In [2]:
import tensorflow as tf
import os
import numpy as np
import random
import tensorflow_io as tfio

seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

2022-12-19 21:37:23.794892: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-19 21:37:23.864931: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-19 21:37:23.868174: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-19 21:37:23.868184: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

## Define Hyper-Parameters

In [3]:
frame_length_in_s = 0.04

PREPROCESSING_ARGS = {
    'downsampling_rate': 16000,
    'frame_length_in_s': frame_length_in_s,
    'frame_step_in_s': frame_length_in_s,
}

TRAINING_ARGS = {
    'batch_size': 32,
    'initial_learning_rate': 0.015,
    'end_learning_rate': 1.e-9,
    'epochs': 60
}


final_sparsity = 0.5
alpha=0.14

num_mel_bins = (int) ((16000 - 16000 * PREPROCESSING_ARGS['frame_length_in_s'])/(16000*PREPROCESSING_ARGS['frame_step_in_s']))+1
print(num_mel_bins)

PREPROCESSING_ARGS = {
    **PREPROCESSING_ARGS,
    'num_mel_bins': num_mel_bins,
    'lower_frequency': 20,
    'upper_frequency': 4000,
}

LABELS = ['down', 'go', 'left', 'no', 'right', 'stop', 'up', 'yes']

downsampling_rate = PREPROCESSING_ARGS['downsampling_rate']
sampling_rate_int64 = tf.cast(downsampling_rate, tf.int64)
frame_length = int(downsampling_rate * PREPROCESSING_ARGS['frame_length_in_s'])
frame_step = int(downsampling_rate * PREPROCESSING_ARGS['frame_step_in_s'])
num_spectrogram_bins = frame_length // 2 + 1
num_mel_bins = PREPROCESSING_ARGS['num_mel_bins']
lower_frequency = PREPROCESSING_ARGS['lower_frequency']
upper_frequency = PREPROCESSING_ARGS['upper_frequency']

25


2022-12-19 21:37:25.043528: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-19 21:37:25.044072: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-19 21:37:25.044195: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-19 21:37:25.044278: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-19 21:37:25.044354: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

## Create Train/Val/Test Datasets

In [4]:
train_ds = tf.data.Dataset.list_files(['msc-train/go*', 'msc-train/stop*'])
val_ds = tf.data.Dataset.list_files(['msc-val/go*', 'msc-val/stop*'])
test_ds = tf.data.Dataset.list_files(['msc-test/go*', 'msc-test/stop*'])


In [5]:
linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
    num_mel_bins=num_mel_bins,
    num_spectrogram_bins=num_spectrogram_bins,
    sample_rate=downsampling_rate,
    lower_edge_hertz=lower_frequency,
    upper_edge_hertz=upper_frequency
)

In [6]:

def preprocess(filename):
    
    audio_binary = tf.io.read_file(filename)

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    true_label = file_parts[0]
    label_id = tf.argmax(true_label == LABELS)

    audio, sampling_rate = tf.audio.decode_wav(audio_binary)
    audio = tf.squeeze(audio)

    zero_padding = tf.zeros(sampling_rate - tf.shape(audio), dtype=tf.float32)
    audio_padded = tf.concat([audio, zero_padding], axis=0)

    stft = tf.signal.stft(
        audio_padded,
        frame_length=frame_length,
        frame_step=frame_step,
        fft_length=frame_length
    )
    spectrogram = tf.abs(stft)
    mel_spectrogram = tf.matmul(spectrogram, linear_to_mel_weight_matrix)
    log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
    log_mel_spectrogram = tf.expand_dims(log_mel_spectrogram, -1)  # channel axis
    mfcss = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)

    return mfcss, label_id

batch_size = TRAINING_ARGS['batch_size']
epochs = TRAINING_ARGS['epochs']

train_ds = train_ds.map(preprocess).batch(batch_size).cache()
val_ds = val_ds.map(preprocess).batch(batch_size)
test_ds = test_ds.map(preprocess).batch(batch_size)

In [7]:
for example_batch, example_labels in train_ds.take(1):
  print('Batch Shape:', example_batch.shape)
  print('Data Shape:', example_batch.shape[1:])
  print('Labels:', example_labels)

Batch Shape: (32, 25, 25, 1)
Data Shape: (25, 25, 1)
Labels: tf.Tensor([1 5 1 5 5 1 5 1 5 1 5 5 1 5 5 1 1 5 1 1 5 1 5 5 5 1 5 1 1 5 5 5], shape=(32,), dtype=int64)


2022-12-19 21:37:25.835512: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


## Create the Model

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=int(128 * alpha), kernel_size=[3, 3], strides=[2, 2], use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=int(128 * alpha), kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=int(128 * alpha), kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=len(LABELS)),
    tf.keras.layers.Softmax()
])

In [9]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

begin_step = int(len(train_ds) * epochs * 0.2)
end_step = int(len(train_ds) * epochs)

pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.40,
        final_sparsity=final_sparsity,
        begin_step=begin_step,
        end_step=end_step
    )
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d   (None, 12, 12, 17)       308       
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_batch_n  (None, 12, 12, 17)       69        
 ormalization (PruneLowMagni                                     
 tude)                                                           
                                                                 
 prune_low_magnitude_re_lu (  (None, 12, 12, 17)       1         
 PruneLowMagnitude)                                              
                                                                 
 prune_low_magnitude_conv2d_  (None, 12, 12, 17)       5204      
 1 (PruneLowMagnitude)                                           
                                                        

In [10]:
linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
    num_mel_bins=num_mel_bins,
    num_spectrogram_bins=num_spectrogram_bins,
    sample_rate=downsampling_rate,
    lower_edge_hertz=lower_frequency,
    upper_edge_hertz=upper_frequency
)

## Train the Mode

In [11]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
initial_learning_rate = TRAINING_ARGS['initial_learning_rate']
end_learning_rate = TRAINING_ARGS['end_learning_rate']

linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=initial_learning_rate,
    end_learning_rate=end_learning_rate,
    decay_steps=len(train_ds) * epochs,
)
optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
metrics = [tf.metrics.SparseCategoricalAccuracy()]
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]
model_for_pruning.compile(loss=loss, optimizer=optimizer, metrics=metrics)

history = model_for_pruning.fit(train_ds, epochs=epochs, validation_data=val_ds,callbacks=callbacks)



Epoch 1/60
50/50 [==============================] - 3s 23ms/step - loss: 0.7715 - sparse_categorical_accuracy: 0.6550 - val_loss: 1.2411 - val_sparse_categorical_accuracy: 0.6200
Epoch 2/60
50/50 [==============================] - 0s 9ms/step - loss: 0.4042 - sparse_categorical_accuracy: 0.8281 - val_loss: 0.8201 - val_sparse_categorical_accuracy: 0.7900
Epoch 3/60
50/50 [==============================] - 0s 8ms/step - loss: 0.3868 - sparse_categorical_accuracy: 0.8363 - val_loss: 1.6700 - val_sparse_categorical_accuracy: 0.6400
Epoch 4/60
50/50 [==============================] - 0s 9ms/step - loss: 0.3343 - sparse_categorical_accuracy: 0.8612 - val_loss: 0.4783 - val_sparse_categorical_accuracy: 0.8300
Epoch 5/60
50/50 [==============================] - 0s 9ms/step - loss: 0.2676 - sparse_categorical_accuracy: 0.8894 - val_loss: 0.3327 - val_sparse_categorical_accuracy: 0.8950
Epoch 6/60
50/50 [==============================] - 1s 10ms/step - loss: 0.2227 - sparse_categorical_accuracy

Epoch 47/60
50/50 [==============================] - 0s 9ms/step - loss: 0.0158 - sparse_categorical_accuracy: 0.9956 - val_loss: 0.2566 - val_sparse_categorical_accuracy: 0.9550
Epoch 48/60
50/50 [==============================] - 0s 9ms/step - loss: 0.0142 - sparse_categorical_accuracy: 0.9962 - val_loss: 0.2624 - val_sparse_categorical_accuracy: 0.9500
Epoch 49/60
50/50 [==============================] - 0s 9ms/step - loss: 0.0132 - sparse_categorical_accuracy: 0.9962 - val_loss: 0.2653 - val_sparse_categorical_accuracy: 0.9500
Epoch 50/60
50/50 [==============================] - 0s 9ms/step - loss: 0.0126 - sparse_categorical_accuracy: 0.9969 - val_loss: 0.2610 - val_sparse_categorical_accuracy: 0.9500
Epoch 51/60
50/50 [==============================] - 0s 9ms/step - loss: 0.0124 - sparse_categorical_accuracy: 0.9969 - val_loss: 0.2548 - val_sparse_categorical_accuracy: 0.9550
Epoch 52/60
50/50 [==============================] - 0s 8ms/step - loss: 0.0124 - sparse_categorical_accu

In [12]:
test_loss, test_accuracy = model_for_pruning.evaluate(test_ds)

training_loss = history.history['loss'][-1]
training_accuracy = history.history['sparse_categorical_accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]

print(f'Training Loss: {training_loss:.4f}')
print(f'Training Accuracy: {training_accuracy*100.:.2f}%')
print()
print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_accuracy*100.:.2f}%')
print()
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy*100.:.2f}%')

7/7 [==============================] - 0s 11ms/step - loss: 0.0938 - sparse_categorical_accuracy: 0.9750
Training Loss: 0.0104
Training Accuracy: 99.69%

Validation Loss: 0.2212
Validation Accuracy: 95.00%

Test Loss: 0.0938
Test Accuracy: 97.50%


## TF model save

In [13]:
from time import time

timestamp = int(time())
modelName = 'model13'

saved_model_dir = f'./saved_models/{modelName}'
if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)
model.save(saved_model_dir)

INFO:tensorflow:Assets written to: ./saved_models/model13/assets


INFO:tensorflow:Assets written to: ./saved_models/model13/assets


## TF convert

In [14]:
MODEL_NAME = modelName

converter = tf.lite.TFLiteConverter.from_saved_model(f'./saved_models/{MODEL_NAME}')
tflite_model = converter.convert()
tflite_models_dir = './tflite_models'
if not os.path.exists(tflite_models_dir):
    os.makedirs(tflite_models_dir)
tflite_model_name = os.path.join(tflite_models_dir, f'{MODEL_NAME}.tflite')
tflite_model_name
with open(tflite_model_name, 'wb') as fp:
    fp.write(tflite_model)

import zipfile

with zipfile.ZipFile(f'{tflite_model_name}.zip', 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(tflite_model_name)

pruned_tflite_size = os.path.getsize(tflite_model_name) / 1024
pruned_zip_size = os.path.getsize(f'{tflite_model_name}.zip') / 1024

print(f'Original TFLite Size (pruned model): {pruned_tflite_size:.2f} KB')
print(f'ZIP TFLite Size (pruned model): {pruned_zip_size:.2f} KB')

Original TFLite Size (pruned model): 24.73 KB
ZIP TFLite Size (pruned model): 14.10 KB


2022-12-19 21:37:57.414014: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-19 21:37:57.414033: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-19 21:37:57.414421: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ./saved_models/model13
2022-12-19 21:37:57.416020: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-19 21:37:57.416041: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ./saved_models/model13
2022-12-19 21:37:57.420191: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-12-19 21:37:57.421305: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-19 21:37:57.449289: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: ./saved_mode

In [15]:
interpreter = tf.lite.Interpreter(model_path=f'./tflite_models/{MODEL_NAME}.tflite')
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print('Number of inputs:', len(input_details))
print('Input name:', input_details[0]['name'])
print('Input index:', input_details[0]['index'])
print('Number of output:', len(output_details))
print('Output name:', output_details[0]['name'])
print('Output index:', output_details[0]['index'])

Number of inputs: 1
Input name: serving_default_input_1:0
Input index: 0
Number of output: 1
Output name: StatefulPartitionedCall:0
Output index: 15


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [16]:
from glob import glob

filenames = glob('msc-test/go*') + glob('msc-test/stop*')
print(len(filenames))

200


In [17]:
from time import time


avg_preprocessing_latency = 0
avg_model_latency = 0
latencies = []
accuracy = 0.0

for filename in filenames:
    audio_binary = tf.io.read_file(filename)

    # NEED ONLY FOR TESTING
    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    true_label = file_parts[0]
    true_label = true_label.numpy().decode()

    # PRE-PROCESSING (LOG-MEL SPECTROGRAM)
    start_preprocess = time()
    audio, sampling_rate = tf.audio.decode_wav(audio_binary)
    audio = tf.squeeze(audio)

    zero_padding = tf.zeros(sampling_rate - tf.shape(audio), dtype=tf.float32)
    audio_padded = tf.concat([audio, zero_padding], axis=0)

    if downsampling_rate != sampling_rate:
        audio_padded = tfio.audio.resample(audio_padded, sampling_rate_int64, downsampling_rate)

    stft = tf.signal.stft(
        audio_padded,
        frame_length=frame_length,
        frame_step=frame_step,
        fft_length=frame_length
    )
    spectrogram = tf.abs(stft)
    mel_spectrogram = tf.matmul(spectrogram, linear_to_mel_weight_matrix)
    log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
    log_mel_spectrogram = tf.expand_dims(log_mel_spectrogram, 0)  # batch axis
    log_mel_spectrogram = tf.expand_dims(log_mel_spectrogram, -1)  # channel axis
    mfcss = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)
    end_preprocess = time()
    interpreter.set_tensor(input_details[0]['index'], mfcss)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    end_inference = time()

    top_index = np.argmax(output[0])
    predicted_label = LABELS[top_index]

    accuracy += true_label == predicted_label
    avg_preprocessing_latency += (end_preprocess - start_preprocess)
    avg_model_latency += (end_inference - end_preprocess)
    latencies.append(end_inference - start_preprocess)

In [18]:
accuracy /= len(filenames)
avg_preprocessing_latency /= len(filenames)
avg_model_latency /= len(filenames)
total_latency = np.median(latencies)

import os

model_size = os.path.getsize(f'./tflite_models/{MODEL_NAME}.tflite')

print(f'Accuracy: {accuracy * 100.:.3f}%')
print(f'Model size: {model_size / 1024:.1f} KB')
print(f'Preprocessing Latency: {1000 * avg_preprocessing_latency:.1f} ms')
print(f'Model Latency: {1000 * avg_model_latency:.1f} ms')
print(f'Total Latency: {1000 * total_latency:.1f} ms')

Accuracy: 97.500%
Model size: 24.7 KB
Preprocessing Latency: 5.1 ms
Model Latency: 0.1 ms
Total Latency: 4.9 ms


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1a21d5cf-232d-49da-b53d-f1e5025e08e5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>